In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
import resource
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

import os
from os import listdir
from os.path import join, isdir
from timeit import default_timer as timer
from scipy import ndimage as ndi
#%%

mypath='/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4'
# mypath='/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-1_Col-9_20230222/FLT_IMG_DIR'

# onlyfiles = [join(mypath, f) for f in listdir(mypath) if isdir(join(mypath, f))]
onlyfiles = [join(mypath, f) for f in listdir(mypath)]
onlyfiles.sort()
#del onlyfiles[-1]# remove cube file from list
onlyfiles_len=len(onlyfiles)

flt_cube_list=[]
int_cube_list=[]

for tile_file in range(onlyfiles_len):
    mat_fname=onlyfiles[tile_file]
    print(mat_fname)
    mat_contents = sio.loadmat(mat_fname)
    img_flt_ref=mat_contents['img_flt']
    img_int_ref=mat_contents['img_int']
    img_flt=img_flt_ref[()]
    img_int=img_int_ref[()]
    flt_cube_list.append(img_flt)
    int_cube_list.append(img_int)
#%%
cube_siz=np.shape(flt_cube_list[0])
cube_row=cube_siz[0]*3
cube_col=cube_siz[1]*3
cube_page=cube_siz[2]

cube_flt=np.zeros((cube_row,cube_col,cube_page))

cube_int=np.zeros((cube_row,cube_col,cube_page))

# row_start=[0,0,0,512,512,512,1024,1024,1024]
# row_stop=[512,512,512,1024,1024,1024,1536,1536,1536]
# col_start=[0,512,1024,0,512,1024,0,512,1024]
# col_stop=[512,1024,1536,512,1024,1536,512,1024,1536]
row_start=np.array([0,0,0,512,512,512,1024,1024,1024])
row_stop=np.array([512,512,512,1024,1024,1024,1536,1536,1536])
col_start=np.array([0,512,1024,0,512,1024,0,512,1024])
col_stop=np.array([512,1024,1536,512,1024,1536,512,1024,1536])

a=np.array([1,4,7,2,5,8,3,6,9])
ai=np.argsort(a)
row_start=row_start[ai]
row_stop=row_stop[ai]
col_start=col_start[ai]
col_stop=col_stop[ai]

for tile_index in range(onlyfiles_len):
    print('Col start:%s Col stop:%s'%(col_start[tile_index],col_stop[tile_index]))
    print('Row start:%s Row stop:%s'%(row_start[tile_index],row_stop[tile_index]))
    flt_img_tile=flt_cube_list[tile_index]
    int_img_tile=int_cube_list[tile_index]
    cube_flt[row_start[tile_index]:row_stop[tile_index],col_start[tile_index]:col_stop[tile_index],:]=flt_img_tile
    cube_int[row_start[tile_index]:row_stop[tile_index],col_start[tile_index]:col_stop[tile_index],:]=int_img_tile

# matfile_filename=mypath+'/'+'cube.mat'
# mdic={"cube":cube}
# sio.savemat(matfile_filename, mdic)
#%%
cube_int=ndi.rotate(cube_int,90)
cube_flt=ndi.rotate(cube_flt, 90)

#for page in range(cube_page):
#    cube_int[:,:,page]=np.fliplr(cube_int[:,:,page])
#    cube_flt[:,:,page]=np.fliplr(cube_flt[:,:,page])
#cube_flt[cube_flt<0]=0
#cube_flt[cube_flt>25]=25
for page in range(cube_page):
    cube_int_slice=np.fliplr(cube_int[:,:,page])
    SNR=np.sqrt(np.mean(cube_int_slice))
    cube_int_slice[cube_int_slice<SNR]=0
    cube_int_slice[cube_int_slice<0]=0
    cube_int[:,:,page]=cube_int_slice
    
    cube_flt_slice=np.fliplr(cube_flt[:,:,page])
    cube_flt_slice[cube_int_slice<SNR]=0
    cube_flt_slice[cube_int_slice<0]=0
    cube_flt_slice[cube_flt_slice<0]=0
    cube_flt_slice[cube_flt_slice>5]=5
    cube_flt[:,:,page]=cube_flt_slice
print('Done')

/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/0.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/1.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/2.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/3.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/4.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/5.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/6.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/7.mat
/home/cruk/Documents/PyWS_CRUK/CRUK_Image_Analysis/Test_Data/Tumour/Row-5_Col-11_20230224/FLT_IMG_DIR_4/8.mat
Col start:

In [2]:
import ipywidgets as ipyw
%matplotlib inline
#%%

class ImageSliceViewer3D:
    """ 
    ImageSliceViewer3D is for viewing volumetric image slices in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image and 
    the slice plane being viewed. 

    Argumentss:
    Volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('plasma'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/users/colormaps.html
    
    """
    
    def __init__(self, volume, figsize=(8,8), cmap='gray'):
        self.volume = volume
        self.figsize = figsize
        self.cmap = cmap
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane selection
        orient = {"y-z":[1,2,0], "z-x":[2,0,1], "x-y": [0,1,2]}
        self.vol = np.transpose(self.volume, orient[view])
        maxZ = self.vol.shape[2] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        self.fig = plt.figure(figsize=self.figsize)
        plt.imshow(self.vol[:,:,z], cmap=plt.get_cmap(self.cmap), 
            vmin=self.v[0], vmax=self.v[1])
        plt.colorbar()

In [3]:
ImageSliceViewer3D(cube_int)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [4]:
ImageSliceViewer3D(cube_flt)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [ ]:
cube_intX=cube_int
cube_fltX=cube_flt

In [ ]:
SNR=np.sqrt(np.mean(cube_intX))
print(SNR)

In [ ]:
cube_intX[cube_intX<SNR]=0
print('Done')

In [ ]:
ImageSliceViewer3D(cube_intX)

In [ ]:
cube_fltX[cube_intX<SNR]=0
print('Done')

In [ ]:
ImageSliceViewer3D(cube_fltX)